In [4]:
from IPython.display import Latex
import numpy as np

# 常用numpy函数
## expand_dims
在指定位置插入新的轴来扩展数组形状

例如原本为一维的2个数据，axis=0，则shape变为(1,2),axis=1则shape变为(2,1)
再例如 原本为 (2,3),axis=0，则shape变为(1,2,3),axis=1则shape变为(2,1,3)

In [6]:
# 测试expand_dims函数
x = np.array([1,2,3])
print(x)
print("x.shape: ",x.shape)
y = np.expand_dims(x,axis=0)
print(y)
print("y.shape: ",y.shape)
print("y[0][1]: ",y[0][1])
y = np.expand_dims(x,axis=1)
print(y)
print("y.shape: ",y.shape)
print("y[1][0]: ",y[1][0])

[1 2 3]
x.shape:  (3,)
[[1 2 3]]
y.shape:  (1, 3)
y[0][1]:  2
[[1]
 [2]
 [3]]
y.shape:  (3, 1)
y[1][0]:  2


# 矩阵微积分
## 标量关于向量的偏导数
$$\frac{\partial y}{\partial \boldsymbol{x}}=[\frac{\partial y}{\partial x_1},\dots,\frac{\partial y}{\partial x_M}]^T$$

## 向量关于向量的偏导数
$$\frac{\partial f(\boldsymbol{x})}{\partial \boldsymbol{x}}=\begin{bmatrix} \frac{\partial y_1}{\partial x_1} & \dots  & \frac{\partial y_N}{\partial x_1} \\
\vdots & \vdots & \vdots \\
 \frac{\partial y_1}{\partial x_M} & \dots & \frac{\partial y_N}{\partial x_M}\end{bmatrix}$$

## 向量函数及其导数
$$\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}} = \boldsymbol{I}（单位矩阵）$$
$$\frac{\partial \boldsymbol{Ax}}{\partial \boldsymbol{x}}=\boldsymbol{A}^T $$
$$\frac{\partial \boldsymbol{x^TA}}{\partial \boldsymbol{x}}=\boldsymbol{A} \\$$